# Mount to drive - Majd:


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/eCornell"

Mounted at /content/drive
/content/drive/MyDrive/eCornell


# Mount to drive - Charmi:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/csv_files"

Mounted at /content/drive
/content/drive/MyDrive/csv_files


In [4]:
# Mount to drive - Eric:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/csv_files"

ModuleNotFoundError: No module named 'google.colab'

# Install required libraries:

In [ ]:
!pip install llama_index
!pip install openai
!pip install fastapi
%pip install llama-index-vector-stores-chroma
!pip install llama-index-readers-file
!pip install llama-index-readers-file
%env OPENAI_API_KEY =sk-proj-nsVQM7joUIQl7GjFvUMZT3BlbkFJhR3MR3VoMb1nWnXRR1PC
!pip install llama_index.experimental
!pip install llama-index.readers.database
!pip install fuzzywuzzy

INFO: pip is looking at multiple versions of llama-index-core to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of llama-index-legacy to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.3/361.3 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 1

# ChatBot

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.core import SimpleDirectoryReader
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
from llama_index.readers.file import (CSVReader, PandasCSVReader)
import httpx
OPENAI_API_KEY ='sk-proj-nsVQM7joUIQl7GjFvUMZT3BlbkFJhR3MR3VoMb1nWnXRR1PC'
from llama_index.llms.openai import OpenAI
import pandas as pd
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.llms.openai import OpenAI
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.experimental.query_engine import PandasQueryEngine
import pandas as pd
import nest_asyncio
from llama_index.agent.openai import OpenAIAgent
import sys
from difflib import get_close_matches
import logging

logging.basicConfig(stream=sys.stdout, level=logging.WARN)
nest_asyncio.apply()

# Set to True if you want to extract the skills again
ex_skills = False
temp = 0.1
is_verbose = False

# Vanilla AI for simple tasks:
import openai

def query_open_ai(prompt, temperature=temp):
    completions = openai.chat.completions.create(
        model="gpt-4",
        messages=[
            {
                "role": "system",
                "content": prompt
            }
        ],
        temperature=temperature,
    )

    return completions.choices[0].message.content

if ex_skills:
  # Load the datasets
  certificates_df = pd.read_csv("./certificates.csv")
  courses_df = pd.read_csv("./updated_courses.csv")
  # # Extract and add skills to DataFrames


  # # max_tokens= 4000
  # def extract_skills(title, text, certificate, domain):
  #     prompt = f" Think like a seasoned career advisor. Given these takeaways for the eCornell online course: \
  #      '{title}', which is part of the certifcate(s): '{certificate}', \
  #      in the domain: '{domain}' give the top 5 skills that are trending resume keywords. \
  #      the skills should be concise with 4 words or less unless needed. the skills should be achieved in the course \
  #       This is a list of the takeaways: '{text}'."
  #     response = query_open_ai(prompt)
  #     skills = [line.strip() for line in response.split('\n') if line.strip()]
  #     return ', '.join(skills)

  # def add_skills_to_courses(df):
  #     skills_list = []
  #     counter = 0
  #     for _, row in df.iterrows():
  #         skills = extract_skills(row['title'], row['takeaways'], row['certificate'], row['domain'])
  #         skills_list.append(skills)
  #         if counter%50 == 0:
  #             print(row['title'], skills_list[-1])
  #         counter += 1
  #     df['skills'] = skills_list
  #     return df

  def add_skills_to_certificates(certificates_df, courses_df):
      skills_list = []
      # counter = 0
      for _, row in certificates_df.iterrows():
          # if counter%30 == 0:
          #     print(row['title'], skills_list[-1])
          # counter += 1
          course_titles = row['courses'].split(', ')
          skills = set()
          for raw_title in course_titles:
              title = ''.join([i.lower() for i in raw_title if i.isalpha() or i == ' ']).strip()
              course_skills = courses_df[courses_df['title'] == title]['skills'].values
              if len(course_skills) > 0:
                  skills.update(course_skills[0].split(', '))
          skills_list.append(', '.join(skills))
      certificates_df['skills'] = skills_list
      return certificates_df

  # courses_df = add_skills_to_courses(courses_df)
  # courses_df.to_csv("./updated_courses.csv", index=False)

  certificates_df = add_skills_to_certificates(certificates_df, courses_df)
  certificates_df.to_csv("./updated_certificates.csv", index=False)


# Load the datasets
certificates_df = pd.read_csv("./updated_certificates.csv")
courses_df = pd.read_csv("./updated_courses.csv")

def clean_dataframe(df):
    df = df.dropna(how='any')  # or df = df.fillna('') to fill NaNs with empty strings
    return df


# Setting up different index for certificates and courses
documents = ['certificates', 'courses']
index_set = {'certificates': clean_dataframe(certificates_df), 'courses': clean_dataframe(courses_df)}




# Convert DataFrames to query engines: https://docs.llamaindex.ai/en/stable/examples/query_engine/pandas_query_engine/
# Convert DataFrames to query engines: https://docs.llamaindex.ai/en/stable/examples/query_engine/pandas_query_engine/

query_engines = {}
for document in documents:

    # index_set[document] = clean_dataframe(index_set[document])  # Clean the DataFrame before creating the query engine
    query_engines[document] = PandasQueryEngine(df=index_set[document], verbose=is_verbose)

individual_query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engines[document],
        metadata=ToolMetadata(
            name= f'vector_index_{document}',
            description=f"Interact as eCornell Chat bot assistant. You will be useful to answer \
            queries about the {document} offered by eCornell",
        )
    )
    for document in documents
]

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=individual_query_engine_tools,
        # define LLM
    llm = OpenAI(temperature=temp, model="gpt-4"),
    verbose=is_verbose,
)


query_engine_tool = QueryEngineTool(
    query_engine=query_engine,
    metadata=ToolMetadata(
        name="general_query_engine",
        description="Interact as eCornell Chat bot assistant. You will be useful \
         when want to answer queries that require analyzing certificates or courses \
         documents available on eCornell platform.",
    ),
)

tools = individual_query_engine_tools + [query_engine_tool]

agent = OpenAIAgent.from_tools(tools, verbose=is_verbose)



# Function to prompt the user to pick a purpose
def get_user_purpose():
    print("""Chatbot: Hi! I'm EZRA (Easy Recommendation Assistant), and I'm here
    to assist you in finding the perfect professional development program to meet
    your needs because we want your to succeed. Let's get started!""")
    print("What is the purpose of your visit today?")
    print("1. Upskill")
    print("2. Transition to a new career")
    print("3. Recent graduate")
    print("4. Not sure")
    purpose = input("Please select an option (1-4): ")
    return purpose

# Function to get top 4 domains in terms of number of certificates using OpenAI API
def get_top_domains():
    prompt = 'from the domain column in ther certificate dataset, \
    list the 4 domains that have the greatest number of certificates as a numbered list.\
    he listing should be of the form: 1. first_domain /n 2. second_domain /n 3. third_domain / 4. fourth_domain'
    response = agent.chat(prompt).response
    domains = [line.strip() for line in response.split('\n') if line.strip()]
    return domains

# Function to list the domain options
def list_domain_options(top_domains):
    print("Let's help you upskill. Please select a domain from the list of popular options:")
    for d in top_domains[:]:
        print(d)
    print('5. Others')

    domain_selection = input("Please select a domain option (1-5): ")
    if int(domain_selection)  == 5:
      response = agent.chat(('list all domains from the certificate dataset as a numbered list. \
      start this list with the sentnece: \
      Here is a list of all the learning domains offered by eCornell:')).response
      domains = [line.strip() for line in response.split('\n') if line.strip()]
      for domain in domains:
        print(domain)
      domain_selection = input("Please select a domain option (1-16): ")
      return domains[int(domain_selection)]
    else:
      return top_domains[int(domain_selection)-1]

# Function to get top 4 skills using OpenAI API
def get_top_skills(selected_domain):
    # prompt = f" list in a numbered list the most popular 4 skills that could be resume keywords. This skills \
    # should help the costumer get a job in the {selected_domain} domain. \
    # The skills should be extracted from the courses and certificated that have {selected_domain} \
    # as one of their domains using the skills column in the datasets. The skills should not \
    # be too specific. They should be key skills that a costumer is looking to develop through online courses."
    prompt = f"among the skills that contain the '{selected_domain[3:]}' domain in the domain column, \
    choose 4 skills that are the most important to appear on a resume to help the learner get a job. \
    The skills should be read from the skills column of the courses and certificates \
    that str.contain the '{selected_domain[3:]}' domain as one of their domains in the domain column. \
    Use the skills column listed in the datasets. These skills should be broad and \
     essential for career growth through online learning. start with: These are the most popular skills in the {selected_domain[3:]} domain:"
    try:
      response = agent.chat(prompt).response
    except TypeError:
      pass
    skills = [line.strip() for line in response.split('\n') if line.strip()]
    return skills

def revised_get_top_skills(selected_domain):

  skill_set = set()
  filtered_df = courses_df[courses_df['domain'].str.contains(selected_domain[3:])]
  for _, row in filtered_df.iterrows():
    skills = row['skills'].split(', ')
    skill_set.update([i[3:] for i in skills])

  response = query_open_ai(f" The following skills are all the skills learned \
  from all eCornell courses in the {selected_domain} domain combined:{str(skill_set)}.\
   ouput the top trending 4 skills that could work as resume keywords to help the learner get a job in the {selected_domain} domain. \
  return as a numbered list starting with: These are the top trending skills in the {selected_domain} domain:")

  top_skills = [line.strip() for line in response.split('\n') if line.strip()]

  return top_skills


#Function to list skills options with option 5 is working
def list_skill_options(skills):
    for skill in skills:
        print(skill)
    print(f"5. Others (Type in the skill you want to learn)")
    skill_selection = input("Please select a skill option (1-5): ")
    # if skill_selection.isdigit() and int(skill_selection) in range(1, 6):
    if int(skill_selection) > len(skills) or int(skill_selection) == 5:
        custom_skill = input("Please type in the skill you want to learn: ")
        return custom_skill
    return skills[int(skill_selection)-1]
    # else:
    #     return skill_selection

# # Function to list skills options
# def list_skill_options(skills):
#     for skill in skills:
#         print(skill)
#     print(f"5. Others")
#     skill_selection = input("Please select a skill option: ")
#     if int(skill_selection) == len(skills) + 1:
#         custom_skill = input("Please type in the skill you want to learn: ")
#         return custom_skill
#     return skills[int(skill_selection)]


# Function to recommend certificates based on domain and skill
# def recommend_certificates(selected_domain, selected_skill):

#     prompt = f" Recommend list of 4 courses or certificates have '{selected_domain}' \
#     listed in its domain section and that have skills related to '{selected_skill.lower()}'. \
#     depend on the skills column to find recommendations. \
#      Please provide three things: short overview, URL link to the course or certificate, \
#      and top careers associated with each. Ensure the information is directly from the dataset \
#      and unchanged. not that the titles in the datasets are all lower case.\
#      order them by relevance with the most relevant options listed first. "
#     response = agent.chat(prompt).response
#     print("Recommended Certificates:")
#     print(response)

def recommend_certificates(selected_domain, selected_skill):

  def find_matches(df):

    def match_skills(skills):
      if not pd.isnull(skills):
        skill_list = skills.split(', ')
        close_matches = get_close_matches(selected_skill.lower()[3:], [skill.lower()[3:] for skill in skill_list], n=1, cutoff=0.6)
        return len(close_matches) > 0
      else:
        return False

    domain_filter = df['domain'].str.contains(selected_domain[3:].strip(), case=False, na=False)
    skills_filter = df['skills'].str.contains(selected_skill[3:].strip(), case=False, na=False)
    filtered_df = df[domain_filter & skills_filter]

    if filtered_df.empty:
      filtered_df = df[df['domain'].str.contains(selected_domain[3:], case=False, na=False)]
      filtered_df = filtered_df[filtered_df['skills'].apply(match_skills)]

    recommendations = []
    for _, row in filtered_df.iterrows():
        title =query_open_ai(f" output this as a title {row.get('title', 'No Title')}. only output the title with no other info :")
        overview = agent.chat(f" summarize this overview for the user in 3 sentences {row.get('overview', 'No Overview')}. only output the summary with hide other info :").response
        recommendations.append({
            'url': row['url'],
            'overview': overview,
            'who': row['who'],
            'title': title
        })

    return recommendations

  certificate_matches = find_matches(certificates_df)
  course_matches = find_matches(courses_df)
  num_cer = min(2, len(certificate_matches))
  num_courses = min(2, len(course_matches))

  print("Recommended Certificate/s: \n")
  for match_ in certificate_matches[:num_cer]:
      print(f"Title: {match_['title']}")
      print(f"URL: {match_['url']}")
      print(f"Overview: {match_['overview']}")
      print(f"Who Should Enroll:")
      for who in match_['who'][2:-2].split("', '"):
        print('→', who)
      print("-" * 40)

  print("\n\nRecommended Course/s: \n")
  for match_ in course_matches[:num_courses]:
      print(f"Title: {match_['title']}")
      print(f"URL: {match_['url']}")
      print(f"Overview: {match_['overview']}")
      print(f"Who Should Enroll:")
      for who in match_['who'][2:-2].split("', '"):
        print('→', who)
      print("-" * 40)


# The Chatbot Flow loop
while True:
    purpose = get_user_purpose()

    if purpose in ['1', '2', '3']:
        print("Great choice! Let's narrow it down further.\n")

        top_domains = get_top_domains()
        selected_domain = list_domain_options(top_domains)
        print(f"You selected: {selected_domain}. Excellent choice!\n")
        print("Let's narrow it down further. Choose the skills which you would like to focus on: ")

        top_skills = revised_get_top_skills(selected_domain)
        selected_skill = list_skill_options(top_skills)
        print(f"You selected: {selected_skill}. That's a valuable skill!\n")
        print("eCornell wants you to succeed. Here are our recommedations:")

        recommend_certificates(selected_domain, selected_skill)

        user_input = input("Do you have any other queries? (type 'no' to exit): ")
        if user_input.lower() == "no":
            print("Thank you for using the EZRA (Easy Recommendation Assistant). Have a great day!")
            break

        response = agent.chat(user_input).response
        print(f"Agent: {response}")

    elif purpose == '4':
        print("No worries! I'm here to help with any questions you have.")

        user_input = input("Please type your query: ")
        if user_input.lower() == "no":
            print("Thank you for using the EZRA (Easy Recommendation Assistant). Have a great day!")
            break

        response = agent.chat(user_input).response
        print(f"Agent: {response}")

    else:

        print("Invalid selection. Please select a valid option (1-4).")


Chatbot: Hi! I'm EZRA (Easy Recommendation Assistant), and I'm here
    to assist you in finding the perfect professional development program to meet
    your needs because we want your to succeed. Let's get started!
What is the purpose of your visit today?
1. Upskill
2. Transition to a new career
3. Recent graduate
4. Not sure
Please select an option (1-4): 1
Great choice! Let's narrow it down further.

Let's help you upskill. Please select a domain from the list of popular options:
1. Business and Finance
2. Technology
3. Leadership
4. Hospitality
5. Others
Please select a domain option (1-5): 1
You selected: 1. Business and Finance. Excellent choice!

Let's narrow it down further. Choose the skills which you would like to focus on: 
1. AI and Machine Learning
2. Data Analysis and Interpretation
3. Financial Risk Assessment
4. Strategic Decision Making
5. Others (Type in the skill you want to learn)
Please select a skill option (1-5): 1
You selected: 1. AI and Machine Learning. That'

# **Corporate Program Recommender**

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.core import SimpleDirectoryReader
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
from llama_index.readers.file import (CSVReader, PandasCSVReader)
import httpx
OPENAI_API_KEY ='sk-proj-nsVQM7joUIQl7GjFvUMZT3BlbkFJhR3MR3VoMb1nWnXRR1PC'
from llama_index.llms.openai import OpenAI
import pandas as pd
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.llms.openai import OpenAI
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.experimental.query_engine import PandasQueryEngine
import pandas as pd
import nest_asyncio
from llama_index.agent.openai import OpenAIAgent
import sys
from difflib import get_close_matches
import logging
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

logging.basicConfig(stream=sys.stdout, level=logging.WARN)
nest_asyncio.apply()

# Set to True if you want to extract the skills again
ex_skills = False
temp = 0.4
is_verbose = True

# Vanilla AI for simple tasks:
import openai

def query_open_ai(prompt, temperature=temp):
    completions = openai.chat.completions.create(
        model="gpt-4",
        messages=[
            {
                "role": "system",
                "content": prompt
            }
        ],
        temperature=temperature,
    )

    return completions.choices[0].message.content
# Load the datasets
certificates_df = pd.read_csv("./updated_certificates.csv")
courses_df = pd.read_csv("./updated_courses.csv")

def clean_dataframe(df):
    df = df.dropna(how='any')  # or df = df.fillna('') to fill NaNs with empty strings
    return df


# Setting up different index for certificates and courses
documents = ['certificates', 'courses']
index_set = {'certificates': clean_dataframe(certificates_df), 'courses': clean_dataframe(courses_df)}




# Convert DataFrames to query engines: https://docs.llamaindex.ai/en/stable/examples/query_engine/pandas_query_engine/
# Convert DataFrames to query engines: https://docs.llamaindex.ai/en/stable/examples/query_engine/pandas_query_engine/

query_engines = {}
for document in documents:

    # index_set[document] = clean_dataframe(index_set[document])  # Clean the DataFrame before creating the query engine
    query_engines[document] = PandasQueryEngine(df=index_set[document], verbose=is_verbose)

individual_query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engines[document],
        metadata=ToolMetadata(
            name= f'vector_index_{document}',
            description=f"Interact as eCornell Chat bot assistant. You will be useful to answer \
            queries about the {document} offered by eCornell",
        )
    )
    for document in documents
]

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=individual_query_engine_tools,
        # define LLM
    llm = OpenAI(temperature=temp, model="gpt-4"),
    verbose=is_verbose,
)


query_engine_tool = QueryEngineTool(
    query_engine=query_engine,
    metadata=ToolMetadata(
        name="general_query_engine",
        description="Interact as eCornell Chat bot assistant. You will be useful \
         when want to answer queries that require analyzing certificates or courses \
         documents available on eCornell platform.",
    ),
)

tools = individual_query_engine_tools + [query_engine_tool]

agent = OpenAIAgent.from_tools(tools, verbose=is_verbose)

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
# Function to filter similar skills during extraction
def add_unique_skill(skill, unique_skills, threshold=80):
    if not any(fuzz.ratio(skill, existing_skill) > threshold for existing_skill in unique_skills):
        unique_skills.add(skill)

# Function to remove similar skills
def filter_similar_skills(skills, threshold=80):
    filtered_skills = []
    while skills:
        skill = skills.pop(0)
        filtered_skills.append(skill)
        skills = [s for s in skills if fuzz.ratio(skill, s) < threshold]
    return filtered_skills

# Test prompt for technology domain
user_input = """
-audience: IT professionals and technology managers, professionals looking to enhance their technical skills and manage IT projects more effectively
-content: cloud computing fundamentals, cybersecurity principles, data analysis, machine learning basics, project management for IT projects
-skills: cloud computing, cybersecurity, data analysis, machine learning, IT project management
"""

# Get list of domains
domains = certificates_df['domain'].unique()

# Guess the domain based on user input
domain_prompt = f"Given this list of domains: {str(domains)}, what domain does the following course description fall into? The description is: {user_input}. Respond with the domain only."
domain_guess = query_open_ai(domain_prompt).strip()

# Filter the certificates DataFrame based on the guessed domain
filtered_df = certificates_df[certificates_df['domain'].str.contains(domain_guess, case=False)]

# Extract unique skills from the filtered DataFrame, filtering out similar skills
unique_skills = set()
for skills in filtered_df['skills'].dropna().unique():
    if isinstance(skills, str):
        for skill in skills.split(','):
            add_unique_skill(skill.strip()[3:], unique_skills)

unique_skills = list(unique_skills)
filtered_unique_skills = filter_similar_skills(unique_skills)

# Identify relevant skills based on user input
skills_prompt = f"From this list of skills: {str(filtered_unique_skills)}, which skills match the following course description: {user_input}. Respond with the skills only."
relevant_skills = query_open_ai(skills_prompt).strip().split(',')
relevant_skills = [skill.strip().strip("{}'") for skill in relevant_skills]

# Recommend courses based on the identified skills
recommended_courses = courses_df.dropna()[courses_df.dropna()['skills'].apply(lambda x: any(skill in x for skill in relevant_skills))]

if recommended_courses.empty:
    print("No recommended courses found.")
else:

    # If more than 5 courses are recommended, use OpenAI to score them based on their 'who' column
    if len(recommended_courses) > 5:
        audience = user_input.split('-content')[0].replace('-audience:', '').strip()

        def score_course_who(course_who, audience):
            prompt = f"Rate the relevance of the following course description to this target audience: {audience}. Course Description: {user_input}. Respond with a score from 0 to 100."
            score = query_open_ai(prompt).strip()
            try:
                return int(score)
            except ValueError:
                return 0

        recommended_courses['who_score'] = recommended_courses['who'].apply(lambda who: score_course_who(who, audience))
        recommended_courses = recommended_courses.sort_values(by='who_score', ascending=False).head(5)
        recommended_courses = recommended_courses.drop(columns=['who_score'])

print("Top 5 Recommended Courses:")
for _, row in recommended_courses.iterrows():
  print()
  print(f"{row['title']}".title())
  print(f"       {row['url']}")



Top 5 Recommended Courses:

2D Laminar Flows
       https://ecornell.cornell.edu/courses/engineering/2d-laminar-flows/

Innovating With 5G
       https://ecornell.cornell.edu/courses/technology/innovating-with-5g/

Using Predictive Data Analysis
       https://ecornell.cornell.edu/courses/data-science-analytics/using-predictive-data-analysis/

Understanding And Visualizing Data
       https://ecornell.cornell.edu/courses/data-science-analytics/understanding-and-visualizing-data/

Querying Relational Databases
       https://ecornell.cornell.edu/courses/data-science-analytics/querying-relational-databases/


In [ ]:
""

Top 5 Recommended Courses:

Becoming A Powerful Leader
       https://ecornell.cornell.edu/courses/leadership-and-strategic-management/becoming-a-powerful-leader/

Building Guest Loyalty
       https://ecornell.cornell.edu/courses/hospitality-and-foodservice-management/building-guest-loyalty/

Examining The Uniform System Of Accounts For The Lodging Industry Usali
       https://ecornell.cornell.edu/courses/hospitality-and-foodservice-management/examining-the-uniform-system-of-accounts-for-the-lodging-industry-usali/

Introduction To Hotel Operations
       https://ecornell.cornell.edu/courses/hospitality-and-foodservice-management/introduction-to-hotel-operations/

Services Marketing Planning And Management
       https://ecornell.cornell.edu/courses/hospitality-and-foodservice-management/services-marketing-planning-and-management/


In [ ]:
res = agent.chat(""" recommend certificates that somewhat aligns with this description. Do not be very strict with the requirements:
-audience: senior manager to senior director
some manage managers, some do not.
-content around psychological safety, talk effectively, delegate effectively, managerial courage, accountability, lifelong learning
-discipline, driving ownership, growth mindset, feeling empowered and empowering others.
The ouput should be in the following form:
output 1: print a list of certificates based on each skill mentioned in the description above, in the following format on each line: (skill : certificate_link)
output 2: print three certificates with links that consider all the mentioned skills combines vaguely.
""").response
for r in res.split('\n'):
  print(r)

Added user message to memory:  recommend certificates that somewhat aligns with this description. Do not be very strict with the requirements:
-audience: senior manager to senior director
some manage managers, some do not.
-content around psychological safety, talk effectively, delegate effectively, managerial courage, accountability, lifelong learning
-discipline, driving ownership, growth mindset, feeling empowered and empowering others.
The ouput should be in the following form:
output 1: print a list of certificates based on each skill mentioned in the description above, in the following format on each line: (skill : certificate_link)
output 2: print three certificates with links that consider all the mentioned skills combines vaguely.

=== Calling Function ===
Calling function: vector_index_certificates with args: {"input": "psychological safety"}
> Pandas Instructions:
```
df[df['overview'].str.contains('psychological safety')]
```
> Pandas Output:                                

In [ ]:
res = agent.chat(""" recommend certificates that somewhat aligns with this description. Do not be very strict with the requirements:
-audience: senior manager to senior director
some manage managers, some do not.
-list 1: (content around psychological safety- talk effectively- delegate effectively- managerial courage- accountability- lifelong learning)
-list point 2: (discipline- driving ownership- growth mindset- feeling empowered and empowering others)
The ouput should be in the following form:
output 1: print a list of 3 certificates based on each list mentioned in the description above, in the following format on each line: (list : 3 certificate_link)
output 2: print three certificates with links that consider all the mentioned skills combines vaguely.
""").response
for r in res.split('\n'):
  print(r)

Added user message to memory:  recommend certificates that somewhat aligns with this description. Do not be very strict with the requirements:
-audience: senior manager to senior director
some manage managers, some do not.
-list 1: (content around psychological safety- talk effectively- delegate effectively- managerial courage- accountability- lifelong learning)
-list point 2: (discipline- driving ownership- growth mindset- feeling empowered and empowering others)
The ouput should be in the following form:
output 1: print a list of 3 certificates based on each list mentioned in the description above, in the following format on each line: (list : 3 certificate_link)
output 2: print three certificates with links that consider all the mentioned skills combines vaguely.

=== Calling Function ===
Calling function: vector_index_certificates with args: {"input": "psychological safety"}
> Pandas Instructions:
```
df[df['overview'].str.contains('psychological safety', case=False)]
```
> Pandas 

# **NEW APPROACH - Large text for Corporate recommender**

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from llama_index.core import Document, VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.schema import TextNode

# Load CSV data into pandas DataFrames
certificates_df = pd.read_csv("./certificates.csv")
courses_df = pd.read_csv("./courses.csv")

# Drop irrelevant information
certificates_df = certificates_df.drop(columns=["courses links", "faculty"])
courses_df = courses_df.drop(columns=["faculty"])

# Function to convert DataFrame rows to Document objects
def df_to_documents(df, doc_type):
    documents = []
    for index, row in df.iterrows():
        text = row.to_string()  # Convert the row to a string
        metadata = row.to_dict()  # Use the row as metadata
        metadata['type'] = doc_type  # Add document type metadata
        doc = Document(text=text, metadata=metadata)
        documents.append(doc)
    return documents

# Convert DataFrames to Document objects
certificates_documents = df_to_documents(certificates_df, "certificate")
courses_documents = df_to_documents(courses_df, "course")



# Combine all documents
all_documents = certificates_documents + courses_documents

# Define a custom text splitter
text_splitter = SentenceSplitter(chunk_size=900, chunk_overlap=1)

# Create an ingestion pipeline with transformations
pipeline = IngestionPipeline(transformations=[text_splitter])

# Transform the documents into nodes
nodes = pipeline.run(documents=all_documents)

# Index the transformed documents
vector_index = VectorStoreIndex(nodes)
query_engine = vector_index.as_query_engine()


In [ ]:
# Example query (modify as needed)
query = """-audience: hospitality managers and supervisors, professionals looking to enhance their skills in hospitality management and customer service
-content: hospitality management principles, customer service excellence, conflict resolution, leadership in hospitality, guest satisfaction strategies
-skills: hospitality management, customer service, conflict resolution, leadership, guest satisfaction

give titles, clickable URLS/ links, and who should enroll. print in a pretty manner. recommend three core certificates and 6 courses which links and who should enroll
"""
response = query_engine.query(query)
print(response)

- Certificate Title: Hospitality Management 360
  - URL: [Hospitality Management 360 Certificate](https://ecornell.cornell.edu/certificates/hospitality-and-foodservice-management/hospitality-management-360/)
  - Who Should Enroll: Hospitality managers working in hotels and corporate offices, hospitality professionals aspiring to management positions

- Certificate Title: Customer Service Excellence in Hospitality
  - URL: [Customer Service Excellence in Hospitality Certificate](insert URL here)
  - Who Should Enroll: Hospitality managers and supervisors, professionals looking to enhance their skills in hospitality management and customer service

- Certificate Title: Conflict Resolution Strategies for Hospitality Professionals
  - URL: [Conflict Resolution Strategies for Hospitality Professionals Certificate](insert URL here)
  - Who Should Enroll: Hospitality managers and supervisors, professionals looking to enhance their skills in hospitality management and customer service

Courses

# **NEW APPROACH  - retail chatbot**

In [ ]:
from textblob import TextBlob

In [ ]:
import pandas as pd
import logging
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings
from llama_index.core.tools import QueryEngineTool

from dotenv import load_dotenv

load_dotenv()
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool

# Load CSV data into pandas DataFrames
certificates_df = pd.read_csv("./certificates.csv")
courses_df = pd.read_csv("./courses.csv")

# Drop irrelevant information
certificates_df = certificates_df.drop(columns=["courses links", "faculty"])
courses_df = courses_df.drop(columns=["faculty"])

# Convert DataFrames to Documents
def df_to_documents(df, doc_type):
    documents = []
    for index, row in df.iterrows():
        text = row.to_string()  # Convert the row to a string
        metadata = row.to_dict()  # Use the row as metadata
        metadata['type'] = doc_type  # Add document type metadata
        doc = Document(text=text, metadata=metadata)
        documents.append(doc)
    return documents

certificates_documents = df_to_documents(certificates_df, "certificate")
courses_documents = df_to_documents(courses_df, "course")

# Combine all documents
all_documents = certificates_documents + courses_documents

# Define a custom text splitter with a large chunk size to accommodate metadata
text_splitter = SentenceSplitter(chunk_size=900, chunk_overlap=10)

# Create an ingestion pipeline with transformations
pipeline = IngestionPipeline(transformations=[text_splitter])

# Transform the documents into nodes
nodes = pipeline.run(documents=all_documents)

# # Set up basic logging configuration
# logging.basicConfig(level=logging.DEBUG)

# # If using a specific logger, set it up as well
# logger = logging.getLogger("llama_index")
# logger.setLevel(logging.DEBUG)

# Index the transformed documents
index = VectorStoreIndex(nodes)
query_engine = index.as_query_engine(verbose=False)



In [ ]:
# Create Query Engine Tool
query_tool = QueryEngineTool.from_defaults(
    query_engine,
    name="courses_and_certificates",
    description="A RAG engine with information about eCornell's courses and certificates. Always base your responses strictly on the data retrieved from the dataset."
         f" If you cannot find information in the dataset, inform the user that the requested information is not available.")

# Initialize the agent with tools
instructions = """
You are an upbeat, encouraging eCornell enrollment counselor who helps potential students
find the right professional development certificate to help them develop key on the job skills. You should:
- As you progress through the conversation, show excitement.
- Provide the title, url link, and a short overview for every course or certificate you recommend.
- Always base your responses strictly on the data retrieved from the dataset.
- If you cannot find information in the dataset, inform the user that the requested information is not available.
"""


# Add the query tool to the agent
llm = OpenAI(model="gpt-3.5-turbo", temperature=0.2)
agent = ReActAgent.from_tools([query_tool], llm=llm, instructions = instructions, verbose=False)

In [ ]:
class eCornellEnrollmentCounselor:
    def __init__(self):
        self.state = {}
        self.prompt_step = 0
        self.helping_mode = False
        self.prompts = [
            "What is your current role? (You can type 'skip' to skip any question)",
            "What are some of your responsibilities in your role? (You can type 'skip' to skip any question)",
            "What skills would you like to further develop? (You can type 'skip' to skip any question)",
            "What do you already know about this topic? (You can type 'skip' to skip any question)",
            "What is your budget for your professional development? (You can type 'skip' to skip any question)",
            "If you have tuition reimbursement or assistance, what does that process look like? (You can type 'skip' to skip any question)",
            "Who else will be making this decision with you? (You can type 'skip' to skip any question)",
            "When are you considering starting a professional development program? (You can type 'skip' to skip any question)",
            "What is driving that timeline? (You can type 'skip' to skip any question)"
        ]
        self.current_prompt = self.prompts[self.prompt_step]

    def get_response(self, user_input):

        if user_input.lower() in ['stop', 'quit']:
            return "Thank you for using the eCornell AI assistant. Goodbye!"

        if user_input.lower() == 'help':
            self.helping_mode = True
            return self.current_prompt

        if user_input.lower() == 'exit help':
            self.helping_mode = False
            return "Exited helping mode. You can ask me anything about our certificates and courses. Type 'help' if you need further assistance."

        if self.helping_mode:
            if self.prompt_step == 1 and user_input.lower() != 'skip':
              self.process_response(user_input)
              return self.recommend_skills()
            else:
                self.process_response(user_input)
                if self.prompt_step < len(self.prompts):
                    self.current_prompt = self.prompts[self.prompt_step]
                    return self.current_prompt
                else:
                    return self.recommend_program()
        else:
            return self.handle_query(user_input)

    def process_response(self, user_input):
        print()
        if user_input.lower() != 'skip':
            if self.prompt_step == 0:
                self.state['role'] = user_input
            elif self.prompt_step == 1:
                self.state['responsibilities'] = user_input
            elif self.prompt_step == 2:
                if user_input.lower() == 'other':
                    self.current_prompt = "Please type the skills you would like to further develop:"
                    return self.current_prompt
                self.state['desired_skills'] = user_input
            elif self.prompt_step == 3:
                self.state['experience_level'] = user_input
            elif self.prompt_step == 4:
                self.state['budget'] = user_input
            elif self.prompt_step == 5:
                self.state['tuition_assistance'] = user_input
            elif self.prompt_step == 6:
                self.state['decision_makers'] = user_input
            elif self.prompt_step == 7:
                self.state['start_time'] = user_input
            elif self.prompt_step == 8:
                self.state['timeline_reason'] = user_input

        self.prompt_step += 1
        # # Once all inputs are collected, recommend the program
        # if self.prompt_step > 8:
        #     return self.recommend_program()
        # else:
        #     return self.get_prompt()


# ORIGINAL
    def recommend_skills(self):
        role = self.state.get('role', 'professional')
        responsibilities = self.state.get('responsibilities', '')
        query = f"Based on the role '{role}' and responsibilities '{responsibilities}', recommend some trending skills. Make sure they are related to the correct domain."
        response = query_engine.query(query).response
        recommended_skills = response.split('\n')
        skills_text = "What skills would you like to further develop? Here are some suggested skills based on your role and responsibilities:\n" + "\n".join(recommended_skills) + "\nOr type 'other' to specify other skills."
        self.state['desired_skills'] = None  # Reset desired skills to prompt user input
        return skills_text

    # def recommend_skills(self):
    #     role = self.state.get('role', 'professional')
    #     responsibilities = self.state.get('responsibilities', '')
    #     query = f"Based on the role '{role}' and responsibilities '{responsibilities}', recommend some skills from eCornell's offerings. Make sure they are related to the correct domain."
    #     response = query_engine.query(query).response
    #     recommended_skills = [skill.strip() for skill in response.split('\n') if skill.strip()]

    #     # Prompt user with recommended skills
    #     skills_text = "What skills would you like to further develop? Here are some suggested skills based on your role and responsibilities:\n" + "\n".join(recommended_skills) + "\nOr type 'other' to specify other skills."

    #     self.prompt_step += 1  # Move to the next prompt after recommending skills

    #     # Display the skills text to the user
    #     print(skills_text)

    #     # Function to get user input from the console or any other interface
    #     user_input = input("Please enter your response: ").strip()  # Replace with actual method to get user input

    #     # Check if the user entered 'other'
    #     if user_input.lower() == 'other':
    #         # Prompt user for a custom skill
    #         custom_skill = input("Please specify the skill you want to add: ").strip()  # Replace with actual method to get user input
    #         self.state['desired_skills'] = custom_skill
    #         return f"You have specified a custom skill: {custom_skill}. Thank you!"

    #     # Check if the user input is in the list of recommended skills
    #     if user_input in recommended_skills:
    #         self.state['desired_skills'] = user_input
    #         return f"You have selected the skill: {user_input}. Thank you!"
    #     else:
    #         return "The skill you entered is not in the recommended list. Please try again."


    # def recommend_program(self):
    #     desired_skills = self.state.get('desired_skills', 'not specified')
    #     if desired_skills == 'not specified':
    #         return "You need to specify a skill before we can recommend programs."

    #     query = (
    #         f"Find programs related to {desired_skills} "
    #         f"and print with titles, type (certificate or course), URL links, and a short overview."
    #     )
    #     response = query_engine.query(query).response

    #     if not response:
    #         return "I couldn't find any courses or certificates that match your requirements. Please provide more details or check back later."

    #     recommendations = response.split('\n')  # Assuming the response is a newline-separated list of recommendations

    #     # Format recommendations
    #     recommendation_text = "\n\n".join(recommendations)

    #     recap = (
    #         f"Based on what you've shared: Your role is {self.state.get('role', 'not specified')}, "
    #         f"your responsibilities include {self.state.get('responsibilities', 'not specified')}, "
    #         f"you want to develop skills in {desired_skills}, "
    #         f"you already know {self.state.get('prior_knowledge', 'not specified')} about this topic, "
    #         f"your budget is {self.state.get('budget', 'not specified')}, "
    #         f"the decision makers are {self.state.get('decision_makers', 'not specified')}, "
    #         f"you plan to start around {self.state.get('start_time', 'not specified')} because {self.state.get('timeline_reason', 'not specified')}."
    #     )
    #     return f"{recap}\n\nDo I have this all correct?\n\nI recommend the following program(s) based on your requirements:\n\n{recommendation_text}\n\nHow does this sound?"


# ORIGINAL
    def recommend_program(self):

        query = (f"from ecornell offerings, Recommend up to 4 certificates from the datasets for a {self.state.get('role', 'professional')}, "
        f"with responsibilities {self.state.get('responsibilities', 'not specified')}, "
         f"who wants to develop skills in {self.state.get('desired_skills', 'professional development')}. "
         f"and has experience with {self.state.get('experience_level', 'not specified')}, "
         f"each recommended program should have a budget less than or around {self.state.get('budget', 'not specified')} dollars, "
         f"each recommended program should require time less than or around {self.state.get('timeline_reason', 'not specified')}."
         f"provide the title, type (certificate or course), url link, and a short overview for each. print in reader friendly way"
         f"Always base your responses strictly on the data retrieved from the dataset. all titles and links should be come extactly from the dataset"
         f" If you cannot find information in the dataset, you can ouput less than 4 certificates"
        )
        response = query_engine.query(query).response

        if not response:
            return "I couldn't find any courses or certificates that match your requirements. Please provide more details or check back later."

        recommendations = response.split('\n')  # Assuming the response is a newline-separated list of recommendations

        # Format recommendations
        recommendation_text = "\n\n".join(recommendations)

        recap = (
            f"Based on what you've shared: Your role is {self.state.get('role', 'not specified')}, "
            f"your responsibilities include {self.state.get('responsibilities', 'not specified')}, "
            f"you want to develop skills in {self.state.get('desired_skills', 'not specified')}, "
            f"you already know {self.state.get('experience_level', 'not specified')} about this topic, "
            f"your budget is {self.state.get('budget', 'not specified')}, "
            f"the decision makers are {self.state.get('decision_makers', 'not specified')}, "
            f"you plan to start around {self.state.get('start_time', 'not specified')} because {self.state.get('timeline_reason', 'not specified')}."
        )
        return f"{recap}\n\nDo I have this all correct?\n\nI recommend the following program(s) based on your requirements:\n\n{recommendation_text}\n\nHow does this sound?"

    def handle_query(self, user_input):
        query = user_input
        response = query_engine.query(query)
        return response

# Initialize the chatbot
counselor = eCornellEnrollmentCounselor()

# Interactive loop
print("Hello! I'm your AI assistant for eCornell. You can ask me anything about our certificates and courses. Type 'help' if you want to be asked questions to help us better understand your needs. Type 'stop' or 'quit' to end the session.")

while True:
    user_input = input("> ")
    response = counselor.get_response(user_input)
    print(response)
    if response == "Thank you for using the eCornell AI assistant. Goodbye!":
        break

Hello! I'm your AI assistant for eCornell. You can ask me anything about our certificates and courses. Type 'help' if you want to be asked questions to help us better understand your needs. Type 'stop' or 'quit' to end the session.
What is your current role? (You can type 'skip' to skip any question)

What are some of your responsibilities in your role? (You can type 'skip' to skip any question)

What skills would you like to further develop? Here are some suggested skills based on your role and responsibilities:
Some trending skills related to the domain of cybersecurity that would be beneficial for someone in the role of a software engineer (swe) with responsibilities in protective technology include:
1. Proficiency in Systems Security
2. Understanding of Authenticating Machines
3. Knowledge of Authenticating Humans
4. Ability to Design Discretionary Access Control Schemes
5. Familiarity with Mandatory Access Control mechanisms
Or type 'other' to specify other skills.

What do you al

# **Log:** Extra: Don't run it (It's additiobal)

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.core import SimpleDirectoryReader
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
from llama_index.readers.file import (CSVReader, PandasCSVReader)
import httpx
from llama_index.llms.openai import OpenAI
import pandas as pd
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.llms.openai import OpenAI
from llama_index.core.query_engine import SubQuestionQueryEngine
import pandas as pd

ModuleNotFoundError: No module named 'llama_index'

In [ ]:
%pwd

'/content/drive/MyDrive/eCornell'

In [ ]:
#Setting up different index for certificates and courses

documents = ['certificates','courses']
index_set = {}
index_set[documents[0]] = pd.read_csv("./certificates.csv")
index_set[documents[1]] = pd.read_csv("./courses.csv")


In [ ]:
from llama_index.experimental.query_engine import PandasQueryEngine

In [ ]:
def clean_dataframe(df):
    # Fill or drop NaN values as needed
    df = df.dropna(how='any')  # or df = df.fillna('') to fill NaNs with empty strings
    return df



In [ ]:
is_verbose = False
# Convert DataFrames to query engines: https://docs.llamaindex.ai/en/stable/examples/query_engine/pandas_query_engine/
# Convert DataFrames to query engines: https://docs.llamaindex.ai/en/stable/examples/query_engine/pandas_query_engine/
query_engines = {}
for document in documents:
    index_set[document] = clean_dataframe(index_set[document])  # Clean the DataFrame before creating the query engine
    query_engines[document] = PandasQueryEngine(df=index_set[document], verbose=is_verbose)
# individual_query_engine_tools = [
#     QueryEngineTool(
#         index_set.as_query_engine(),
#         metadata=ToolMetadata(
#             name= "cert_course_query",
#             description=f"You are a chatbot for eCornell, Cornell University's online learning subsidiary, providing professional development courses in various fields. Your main task is to guide users in finding the correct course relevant to their query. When a user gives a query, analyze if it\'s about a course, then respond with a well-formatted list of the top three most relevant courses by mentioning: Short overview, Course Title, URL, and Potential Careers based on the available courses in the uploaded dataset csv files under column 'title'. '
#            ),
#     )
# ]
individual_query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engines[document],
        metadata=ToolMetadata(
            name=f"vector_index_{document}",
            description=f"Interact as eCornell Chat bot assistant. You will be useful to answer \
            queries about the {document} offered by eCornell",
        )
    )
    for document in documents
]

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=individual_query_engine_tools,
        # define LLM
    llm = OpenAI(temperature=0.1, model="gpt-4"),
    verbose=is_verbose,
)

# query_engine_tool = QueryEngineTool(
#     query_engine=query_engine,
#     metadata=ToolMetadata(
#         name="sub_question_query_engine",
#         description=f"You are a chatbot for eCornell, Cornell University's online learning subsidiary, providing professional development courses in various fields. Your main task is to guide users in finding the correct course relevant to their query. When a user gives a query, analyze if it\'s about a course, then respond with a well-formatted list of the top three most relevant courses by mentioning: Short overview, Course Title, URL, and Potential Careers based on the available courses in the uploaded dataset csv files under column 'title'. ",
#         # If the user asks for irrelevant information which won't help model to recommend eCornell courses by any means, respond with: 'I'm a chatbot and will try to find a relevant course for you to help you find the best fit among the available certificates and courses. Please type in your query.'",
#         # description= "You are a chatbot for eCornell, Cornell University's online learning subsidiary, providing professional development courses in various fields. Your main task is to guide users in finding the correct course relevant to their query. When a user gives a query, analyze if it\'s about a course, then respond with a well-formatted list of the top three most relevant courses by mentioning: Short overview, Course Title, URL, and Potential Careers based on the data uploaded in the model. If the user asks for irrelevant information or if no relevant data is found, respond with: 'I'm a chatbot and will try to find a relevant course for you to help you find the best fit among the available certificates and courses. Please type in your query.'"
#         # If a user asks for irrelevant information, respond with: \"I'm a chatbot and will try to find a relevant course for you to help you find the best fit among the available certificates and courses. Please type in your query."

#     ),
# )
query_engine_tool = QueryEngineTool(
    query_engine=query_engine,
    metadata=ToolMetadata(
        name="general_query_engine",
        description="Interact as eCornell Chat bot assistant. You will be useful \
         when want to answer queries that require analyzing certificates or courses \
         documents available on eCornell platform",
    ),
)

tools = individual_query_engine_tools + [query_engine_tool]

import nest_asyncio
nest_asyncio.apply()
from llama_index.agent.openai import OpenAIAgent



In [ ]:
agent = OpenAIAgent.from_tools(tools, max_tokens= 4000, verbose=is_verbose)
# max_tokens= 4000

In [ ]:
def get_user_purpose():
    print("Chatbot: What is the purpose of your visit today?")
    print("1. Upskill")
    print("2. Transition to a new career")
    print("3. Recent graduate")
    print("4. Not sure")
    purpose = input("Please select an option (1-4): ")
    return purpose

def get_top_domains():
    prompt = 'list the 4 domains that have the most certificates as a numbered list. The listing should be of the form: \
    1. first_domain /n 2. second_domain /n 3. third_domain / 4. fourth_domain'
    response = agent.chat(prompt).response
    domains = [line.strip() for line in response.split('\n') if line.strip()]
    return domains

def list_domain_options(top_domains):
    print("Great! Let's help you upskill. Please select a domain from the list of popular options:")
    for d in top_domains[:]:
        print(d)
    print('5. Others')

    domain_selection = input("Please select a domain option (1-5): ")
    if int(domain_selection)  == 5:
      response = agent.chat(('list all domains from the certificate dataset as a numbered list. start this list with the sentnece: \
      Here is a list of all the learning domains offered by eCornell:')).response
      domains = [line.strip() for line in response.split('\n') if line.strip()]
      for domain in domains:
        print(domain)
      domain_selection = input("Please select a domain option (1-17): ")
      return domains[int(domain_selection)]
    else:
      return top_domains[int(domain_selection)-1]

# Function to get top 4 skills using OpenAI API
def get_top_skills(selected_domain):
    prompt = f"Think like a career advisor with 10 years of experience. Use overviews, takeaways, and potential careers \
              to come up with the list of top 4 resume keyword skills associated with '{selected_domain}' \
              the skill should not generally be more than 3 to 4 words maximum unless needed \
               start this list with the sentnece: \
      Here are the top 4 trending skills in the {selected_domain} domain:"
    response = agent.chat(prompt).response
    skills = [line.strip() for line in response.split('\n') if line.strip()]
    return skills

#Function to list skills options with option 5 is working
def list_skill_options(skills):
    for skill in skills:
        print(skill)
    print(f"5. Others (Type in the skill you want to learn)")
    skill_selection = input("Please select a skill option (1-5): ")
    # if skill_selection.isdigit() and int(skill_selection) in range(1, 6):
    if int(skill_selection) == 5:
        custom_skill = input("Please type in the skill you want to learn: ")
        return custom_skill
    return skills[int(skill_selection)]
    # else:
    #     return skill_selection

# # Function to list skills options
# def list_skill_options(skills):
#     for skill in skills:
#         print(skill)
#     print(f"5. Others")
#     skill_selection = input("Please select a skill option: ")
#     if int(skill_selection) == len(skills) + 1:
#         custom_skill = input("Please type in the skill you want to learn: ")
#         return custom_skill
#     return skills[int(skill_selection)]

# Function to recommend certificates based on domain and skill
def recommend_certificates(selected_domain, selected_skill):
    prompt = f" Recommend list of 4 certificates or courses in the '{selected_domain}' domain \
    that are most relevant for gaining skills related to '{selected_skill.lower()}'. \
     Please provide three things: short overview, URL link to the certificate or course, \
     and top careers associated with each. Ensure the information is directly from the dataset \
     and unchanged. Aim to recommend 2 certificates and 2 courses if possible, \
     ordered by relevance with the most relevant options listed first."
    response = agent.chat(prompt).response
    print("Recommended Certificates:")
    print(response)



In [ ]:
while True:
    purpose = get_user_purpose()
    if purpose in ['1', '2', '3']:
        top_domains = get_top_domains()
        selected_domain = list_domain_options(top_domains)
        print(f"You selected: {selected_domain}")
        top_skills = get_top_skills(selected_domain)
        selected_skill = list_skill_options(top_skills)
        print(f"You selected: {selected_skill}")
        recommend_certificates(selected_domain, selected_skill)
        user_input = input("Do you have any other queries? (type 'no' to exit): ")
        if user_input.lower() == "no":
            break
        response = agent.chat(user_input).response
        print(f"Agent: {response}")
    elif purpose == '4':
        user_input = input("Please type your query: ")
        if user_input.lower() == "no":
            break
        response = agent.chat(user_input).response
        print(f"Agent: {response}")
    else:
        print("Invalid selection. Please select a valid option (1-4).")

Chatbot: What is the purpose of your visit today?
1. Upskill
2. Transition to a new career
3. Recent graduate
4. Not sure
Please select an option (1-4): 1
Great! Let's help you upskill. Please select a domain from the list of popular options:
1. Business and Finance
2. Technology
3. Leadership
4. Hospitality
5. Others
Please select a domain option (1-5): 2
You selected: 2. Technology
Here are the top 4 trending skills in the Technology domain:
1. Generative AI for Productivity
2. Designing and Building AI Solutions
3. AI for Digital Transformation
4. AI Strategy
5. Others (Type in the skill you want to learn)
Please select a skill option (1-5): 4
You selected: 4. AI Strategy
Recommended Certificates:
I couldn't find specific certificates related to "AI Strategy" in the Technology domain. However, here are the top 2 courses in the Technology domain:

1. Course: Decision Analysis
   - Overview: Decision analysis is a critical skill for engineers in various fields. This course helps in id

In [ ]:

    prompt = f"Think like a career advisor with 10 years of experience. Use overviews, takeaways, and potential careers \
              to come up with the list of top 4 resume keyword skills associated for \
              the skill should not generally be more than 3 to 4 words maximum unless needed \
               start this list with the sentence: \
      Here are the top 4 trending skills in the {selected_domain} domain:"

In [ ]:
agent.chat(f"Think like a career advisor with 10 years of experience. Use overviews, takeaways, and potential careers from the dataset\
              to come up with the list of top 10 resume keyword skills associated for the eCornell course: 'summarizing and visualizing data' \
              the skill should not generally be more than 3 to 4 words maximum unless needed \
               start this list with the sentnece: ").response

NameError: name 'agent' is not defined

MArketing Team Doc:

In [ ]:
# Initialize the agent
agent = OpenAIAgent.from_tools(individual_query_engine_tools, verbose=False)

# Function to get options from agent's response
def get_options_from_response(response, context=None):
    options = []

    if context == "skills":
        # Example: Extract skills from response (replace with actual logic)
        skills_list = extract_skills(response.response)
        options = skills_list[:3]  # Select the top 3 skills
    elif context == "topics":
        # Example: Extract topics from response (replace with actual logic)
        topics_list = extract_topics(response.response)
        options = topics_list[:3]  # Select the top 3 topics
    elif context == "courses":
        # Example: Extract course titles and URLs from response (replace with actual logic)
        courses_list = extract_courses(response.response)
        options = courses_list[:3]  # Select the top 3 courses
    else:
        # Default case, treat response as plain text options
        options = response.response.split("\n")
        options = [option.strip() for option in options if option.strip()]

        # Handle special options for exit and skip
        if "Skip to next question" in options:
            options.remove("Skip to next question")
        if "Exit" in options:
            options.remove("Exit")

    # Clean up options (remove empty strings and extra spaces)
    options = [option.strip() for option in options if option.strip()]

    return options

# Function to ask a question with options and get the user's choice
def ask_question(question, options, allow_skip=True, allow_exit=True):
    print(f"Chatbot: {question}")
    for idx, option in enumerate(options, 1):
        print(f"{idx}. {option}")
    if allow_skip:
        print(f"{len(options) + 1}. Skip to next question")
    if allow_exit:
        print(f"{len(options) + 2}. Exit")
    choice = input("Your choice: ")

    try:
        choice = int(choice)
    except ValueError:
        choice = -1

    if choice == len(options) + 1 and allow_skip:
        return None  # Skip to next question
    elif choice == len(options) + 2 and allow_exit:
        return "exit"  # Exit the flow
    elif 1 <= choice <= len(options):
        return options[choice - 1]
    else:
        print("Invalid choice. Please select a valid option.")
        return ask_question(question, options, allow_skip, allow_exit)

# Chatbot initiates the conversation
print("Chatbot: Hello! I'm your course recommendation assistant. What's your name?")
user_name = input("User: ")
print(f"Chatbot: Nice to meet you, {user_name}! Let's start with a few questions to help me suggest the best courses for you.")

# Start asking questions
reason = ask_question(
    "What brings you here today?",
    ["Learn something new", "Advance my career", "Get information on a program"],
    allow_skip=False, allow_exit=False
)

learning_about = None
current_position = None
next_step = None
skills_required = None

# Handle the flow based on the reason
if reason == "Learn something new":
    learning_about = ask_question(
        "What are you looking to learn about?",
        ["Data Science", "Leadership", "Marketing"]
    )
elif reason == "Advance my career":
    current_position = ask_question(
        "What's your current position?",
        ["Product Manager", "Software Engineer", "Marketing Specialist"]
    )
    next_step = ask_question(
        f"Where do you want to advance from your current position as a {current_position}?",
        [f"Senior {current_position}", f"Lead {current_position}", f"Director of {current_position}"]
    )
    skills_response = agent.chat(f"What skills are required to advance to a {next_step}?")
    skills_required = get_options_from_response(skills_response, context="skills")
elif reason == "Get information on a program":
    learning_about = ask_question(
        "Which program are you interested in?",
        ["Data Science", "Leadership", "Marketing"]
    )

# Continue collecting information based on the user's responses
if learning_about:
    topics_response = agent.chat(f"Generate topics related to {learning_about}")
    topics = ask_question(
        "Which topics interest you the most?",
        get_options_from_response(topics_response, context="topics")
    )
else:
    topics = None

if next_step:
    skills_response = agent.chat(f"Generate skills related to {next_step}")
    skills = ask_question(
        f"What skills are you hoping to gain to excel in {next_step}?",
        get_options_from_response(skills_response, context="skills")
    )
else:
    skills = None

experience_years = ask_question(
    f"How many years of experience do you have in {learning_about or next_step}?",
    ["0-5", "6-10", "11+"]
)

if reason != "Advance my career":
    current_work = ask_question(
        "What do you currently do for work?",
        ["Software Engineer", "Project Manager", "Marketing Specialist"]
    )

professional_goals_response = agent.chat("Generate professional goals or dream roles related to the user's current field")
professional_goals = ask_question(
    "What are your professional goals or what's your dream role?",
    get_options_from_response(professional_goals_response, context="goals")
)

challenges_response = agent.chat(f"Generate challenges related to {learning_about or next_step}")
challenges = ask_question(
    f"What is your top {learning_about or next_step} challenge?",
    get_options_from_response(challenges_response, context="challenges")
)

# Process the gathered information and provide course recommendations
print("Chatbot: Thank you for the information! Let me find the best courses for you.")

# Construct a query based on the user's responses
query_parts = [
    learning_about or next_step,
    topics,
    skills_required,
    f"{experience_years} years experience",
    current_work if reason != 'Advance my career' else current_position,
    professional_goals,
    challenges
]
query = ", ".join(part for part in query_parts if part)

response = agent.chat(query)

if response:
    print(f"Chatbot: Based on your interests and goals, here are the top three most relevant courses for you:")
    print(response)
else:
    print("Chatbot: I'm a chatbot and will try to find a relevant course for you to help you find the best fit among the available certificates and courses. Please type in your query.")

# Follow-up options
email = input(f"Chatbot: Can we follow up with you later? Let’s keep in touch. Enter your email: ")
talk_to_human = input("Chatbot: Would you like to talk to a human? (y/n) ")
if talk_to_human.lower() == 'y':
    phone_or_chat = input("Chatbot: Please enter your phone number or click the link to launch live chat: ")


Chatbot: Hello! I'm your course recommendation assistant. What's your name?
User: majd
Chatbot: Nice to meet you, majd! Let's start with a few questions to help me suggest the best courses for you.
Chatbot: What brings you here today?
1. Learn something new
2. Advance my career
3. Get information on a program
Your choice: s
Invalid choice. Please select a valid option.
Chatbot: What brings you here today?
1. Learn something new
2. Advance my career
3. Get information on a program
Your choice: 2
Chatbot: What's your current position?
1. Product Manager
2. Software Engineer
3. Marketing Specialist
4. Skip to next question
5. Exit
Your choice: 2
Chatbot: Where do you want to advance from your current position as a Software Engineer?
1. Senior Software Engineer
2. Lead Software Engineer
3. Director of Software Engineer
4. Skip to next question
5. Exit
Your choice: 2


KeyboardInterrupt: 

Experimenting:

## Follow up:

In [ ]:
def generate_follow_up_questions(response_text):
    # prompt = f"Based on the information provided: {response_text}, generate three follow-up questions for the user to ask. But before generating give detailed as to what user has asked and just revert back that if you need any further guidance then here are follow ups"
    prompt = f"Based on the information provided: {response_text},When the user provides their initial query, ask up to three follow-up questions to gain clarity. For each follow-up question, provide a list of options with index numbers. The user will respond with the index number corresponding to their choice."
    follow_up_response = agent.chat(prompt)
    return follow_up_response

chat_engine = index.as_chat_engine()
streaming_response = chat_engine.stream_chat("Tell me a joke.")
for token in streaming_response.response_gen:
    print(token, end="")

while True:
    text_input = input("User: ")
    if text_input.lower() == "exit":
        break
    response = agent.chat(text_input)
    print(f"Agent: {response}")

    follow_up_response = generate_follow_up_questions(str(response))
    print("Follow-Up Suggestions:")
    follow_up_questions = str(follow_up_response).split('\n')
    for idx, question in enumerate(follow_up_questions, 1):
        print(question)

In [ ]:
parser = PandasCSVReader()
file_extractor = {".csv": parser}
documents = SimpleDirectoryReader(input_files=["./certificates.csv", "./courses.csv"], file_extractor=file_extractor).load_data()
index = VectorStoreIndex.from_documents(documents)

In [ ]:
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
)
# query
response = query_engine.query("time, effort, faculty for title how to create interactive websites with html and css?")
print(response)